# YouTube Song URL Finder

This notebook uses the YouTube Data API to find video URLs for songs when given the title and artist name.

In [2]:
# Install required packages
!pip install google-api-python-client python-dotenv

In [1]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
API_KEY = os.getenv("YOUTUBE_API_KEY")

# Verify API key is loaded
if not API_KEY:
    print("WARNING: API_KEY not found in environment variables!")
    print("Please create a .env file with your YOUTUBE_API_KEY=your_key_here")
else:
    print("API key loaded successfully!")

# Create YouTube API client
youtube = build('youtube', 'v3', developerKey=API_KEY)

API key loaded successfully!


In [4]:
def find_song_url(title, artist, max_results=5):
    """
    Search for a song on YouTube and return the top video URLs.
    
    Args:
        title (str): The song title
        artist (str): The artist name
        max_results (int): Maximum number of results to return
        
    Returns:
        list: List of dictionaries containing video titles and URLs
    """
    try:
        # Create the search query
        query = f"{title} {artist} official"
        
        # Call the search.list method to retrieve results
        search_response = youtube.search().list(
            q=query,
            part="id,snippet",
            maxResults=max_results,
            type="video"
        ).execute()
        
        # Collect the results
        videos = []
        for search_result in search_response.get("items", []):
            if search_result["id"]["kind"] == "youtube#video":
                video_id = search_result["id"]["videoId"]
                video_title = search_result["snippet"]["title"]
                video_url = f"https://www.youtube.com/watch?v={video_id}"
                
                videos.append({
                    "title": video_title,
                    "url": video_url
                })
        
        return videos
    
    except HttpError as e:
        print(f"An HTTP error {e.resp.status} occurred:\n{e.content}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [5]:
# Example usage
song_title = "Bohemian Rhapsody"
artist_name = "Queen"

results = find_song_url(song_title, artist_name)

print(f"Results for '{song_title}' by {artist_name}:\n")
for i, video in enumerate(results, 1):
    print(f"{i}. {video['title']}")
    print(f"   URL: {video['url']}\n")

Results for 'Bohemian Rhapsody' by Queen:

1. Queen – Bohemian Rhapsody (Official Video Remastered)
   URL: https://www.youtube.com/watch?v=fJ9rUzIMcZQ

2. Queen - Bohemian Rhapsody (Official Lyric Video)
   URL: https://www.youtube.com/watch?v=jFKBR1ggTMY

3. Queen – Bohemian Rhapsody (Lyrics)
   URL: https://www.youtube.com/watch?v=yk3prd8GER4

4. Queen - Bohemian Rhapsody [Chief Mouse Restoration]
   URL: https://www.youtube.com/watch?v=p1oPAupjFtk

5. Bohemian Rhapsody (Live Aid)
   URL: https://www.youtube.com/watch?v=yvQYV520tS8



## Create a function to search for multiple songs

In [6]:
def find_urls_for_songs(songs_list):
    """
    Find YouTube URLs for a list of songs.
    
    Args:
        songs_list (list): List of dictionaries with 'title' and 'artist' keys
        
    Returns:
        dict: Dictionary with song info and top YouTube URL
    """
    results = {}
    
    for song in songs_list:
        title = song['title']
        artist = song['artist']
        
        print(f"Searching for '{title}' by {artist}...")
        videos = find_song_url(title, artist, max_results=1)
        
        key = f"{title} - {artist}"
        if videos:
            results[key] = {
                'title': title,
                'artist': artist,
                'youtube_title': videos[0]['title'],
                'youtube_url': videos[0]['url']
            }
        else:
            results[key] = {
                'title': title,
                'artist': artist,
                'youtube_title': 'Not found',
                'youtube_url': 'Not found'
            }
    
    return results

In [7]:
# Example with multiple songs
songs = [
    {'title': 'Bohemian Rhapsody', 'artist': 'Queen'},
    {'title': 'Billie Jean', 'artist': 'Michael Jackson'},
    {'title': 'Hotel California', 'artist': 'Eagles'}
]

all_results = find_urls_for_songs(songs)

# Display results in a more structured way
import pandas as pd

df = pd.DataFrame.from_dict(all_results, orient='index')
df

Searching for 'Bohemian Rhapsody' by Queen...
Searching for 'Billie Jean' by Michael Jackson...
Searching for 'Hotel California' by Eagles...


,title,artist,youtube_title,youtube_url
Bohemian Rhapsody - Queen,Bohemian Rhapsody,Queen,Queen – Bohemian Rhapsody (Official Video Rema...,https://www.youtube.com/watch?v=fJ9rUzIMcZQ
Billie Jean - Michael Jackson,Billie Jean,Michael Jackson,Michael Jackson - Billie Jean (Official Video),https://www.youtube.com/watch?v=Zi_XLOBDo_Y
Hotel California - Eagles,Hotel California,Eagles,Eagles - Hotel California (Live 1977) (Officia...,https://www.youtube.com/watch?v=09839DpTctU


## Save results to a file

In [10]:
# Save results to CSV
df.to_csv('new_songs_youtube_urls.csv')
print("Results saved to 'song_youtube_urls.csv'")

Results saved to 'song_youtube_urls.csv'
